In [2]:
import pandas as pd
import cooler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from os import listdir, system, path
import re

In [3]:
!pwd

/tank/projects/polycomb/results/2024.04.23


In [4]:
# Read cell type annotation
ct_ann_path = '/tank/projects/polycomb/data/heffel/supp_table_s3.cell_type_annotation.xlsx'
ct_ann = pd.read_excel(ct_ann_path, skiprows=1)
ct_ann.head()

,Cell ID,L1 annotation,L2 annotation,L3 annotation,L4 annotation,mCCC/CCC,mCH/CH,mCG/CG,Sample,region,age_groups,TotalInt
0,210830-4267-HPC-3-C07_F15,Non-Neuron,Glial-Non-Neuron,OPC,3T_OPC,0.00859,0.01151,0.75400,210830-4267-HPC,HPC,3T,138582
1,210830-4267-HPC-3-C07_C2,Exc,Exc-CA,Exc-CA-TSHZ2-MEIS2,3T_Exc-CA-TSHZ2-MEIS2,0.00927,0.01860,0.75798,210830-4267-HPC,HPC,3T,139219
2,210830-4267-HPC-3-C07_J11,Exc,Exc-ENT,Exc-ENT,NaN,0.00780,0.01168,0.73789,210830-4267-HPC,HPC,3T,194860
3,210830-4267-HPC-3-C07_C24,Non-Neuron,Glial-Non-Neuron,OPC,3T_OPC,0.00815,0.01218,0.72286,210830-4267-HPC,HPC,3T,137466
4,210830-4267-HPC-3-C07_J7,Exc,Exc-CA,Exc-CA1,3T_Exc-CA1,0.00903,0.02066,0.74035,210830-4267-HPC,HPC,3T,203616


In [5]:
ct_ann.shape

(53063, 12)

In [12]:
# List cell types
ct_ann['L1 annotation'].unique()

array(['Non-Neuron', 'Exc', 'Inh', 'RG'], dtype=object)

In [13]:
# List cell types
ct_ann['L2 annotation'].unique()

array(['Glial-Non-Neuron', 'Exc-CA', 'Exc-ENT', 'Inh-MGE', 'Inh-CGE',
       'RG', 'Non-Neuron', 'Exc-DG', 'Exc-UL', 'Exc-DL'], dtype=object)

In [9]:
# List cell types
ct_ann.loc[(ct_ann['region'] == 'CTX') & 
           (ct_ann['L3 annotation'] == 'Astro') & 
           (ct_ann['age_groups'] == 'adult')]

,Cell ID,L1 annotation,L2 annotation,L3 annotation,L4 annotation,mCCC/CCC,mCH/CH,mCG/CG,Sample,region,age_groups,TotalInt
25702,20210111-1863-preAb-PFC-B12_A3,Non-Neuron,Glial-Non-Neuron,Astro,adult_Astro,0.00649,0.01423,0.76558,20210111_UMB1863,CTX,adult,103243
25708,20210111-1863-preAb-PFC-B12_B15,Non-Neuron,Glial-Non-Neuron,Astro,adult_Astro,0.00598,0.02028,0.76390,20210111_UMB1863,CTX,adult,137003
25712,20210111-1863-preAb-PFC-B12_A20,Non-Neuron,Glial-Non-Neuron,Astro,adult_Astro,0.00568,0.01738,0.76913,20210111_UMB1863,CTX,adult,166187
25725,20210111-1863-preAb-PFC-B12_B19,Non-Neuron,Glial-Non-Neuron,Astro,adult_Astro,0.00550,0.01574,0.76610,20210111_UMB1863,CTX,adult,190600
25734,20210111-1863-preAb-PFC-B12_C19,Non-Neuron,Glial-Non-Neuron,Astro,adult_Astro,0.00511,0.01807,0.76507,20210111_UMB1863,CTX,adult,139496
...,...,...,...,...,...,...,...,...,...,...,...,...
53039,SRR9014792,Non-Neuron,Glial-Non-Neuron,Astro,adult_Astro,0.00521,0.01987,0.79864,Lee_UMB5580,CTX,adult,486604
53040,SRR9014779,Non-Neuron,Glial-Non-Neuron,Astro,adult_Astro,0.00506,0.01986,0.79438,Lee_UMB5580,CTX,adult,580870
53043,SRR9014751,Non-Neuron,Glial-Non-Neuron,Astro,adult_Astro,0.00471,0.01698,0.78464,Lee_UMB5580,CTX,adult,485329
53056,SRR9014804,Non-Neuron,Glial-Non-Neuron,Astro,adult_Astro,0.00515,0.01795,0.79577,Lee_UMB5580,CTX,adult,407331


In [16]:
# List cell types
ct_ann.loc[(ct_ann['region'] == 'CTX') & (ct_ann['L2 annotation'] == 'Exc-UL'), 'L3 annotation'].unique()

array(['Exc-L1-3-CUX2', 'Exc-L4-6-LRRK1', 'Exc-L4-RORB',
       'Exc-L5-6-PDZRN4', 'Exc-L1-3-NRXN2', 'Exc-L4-PLCH1',
       'Exc-L4-5-FOXP2', 'Exc-UL-1', 'Exc-L4-5-TOX', 'Exc-UL-2'],
      dtype=object)

In [17]:
# List cell types
ct_ann.loc[(ct_ann['region'] == 'CTX') & (ct_ann['L2 annotation'] == 'Exc-DL'), 'L3 annotation'].unique()

array(['Exc-DL-ASTN1', 'Exc-DL-GRIK4', 'Exc-NP-TSHZ2', 'Exc-DL-1',
       'Exc-DL-2', 'Exc-DL-GRIK4-TSHZ2'], dtype=object)

In [5]:
ct_ann.loc[ct_ann['region'] == 'CTX', ['L3 annotation', 'age_groups']].value_counts().to_frame().reset_index()\
    .pivot(columns='age_groups', index='L3 annotation')

count                      
age_groups              2T     3T   adult infant
L3 annotation                                   
Astro                  7.0  338.0   931.0  496.0
EC                    12.0    9.0   204.0   97.0
Exc-DL-1             639.0    NaN     NaN    NaN
Exc-DL-2             328.0    3.0     NaN    NaN
Exc-DL-ASTN1           2.0  122.0   166.0  102.0
Exc-DL-GRIK4           NaN    NaN   240.0  121.0
Exc-DL-GRIK4-TSHZ2     1.0  127.0     NaN    NaN
Exc-L1-3-CUX2          NaN  930.0  1090.0  572.0
Exc-L1-3-NRXN2         NaN    NaN  1096.0  248.0
Exc-L4-5-FOXP2         NaN    NaN   531.0  138.0
Exc-L4-5-TOX           NaN    NaN   161.0    NaN
Exc-L4-6-LRRK1         NaN    NaN   424.0  335.0
Exc-L4-PLCH1           NaN    NaN   245.0  207.0
Exc-L4-RORB            NaN  635.0   567.0  232.0
Exc-L5-6-PDZRN4        NaN  244.0   353.0  321.0
Exc-NP-TSHZ2           NaN    NaN    79.0   43.0
Exc-UL-1            2333.0   53.0     NaN    9.0
Exc-UL-2               NaN   62.0     NaN    NaN
Inh-CGE                6.0  430.0     NaN    NaN
Inh-CGE-ALK            NaN    NaN    55.0   17.0
Inh-CGE-CHRNA2         NaN    NaN   267.0  165.0
Inh-CGE-CLMP           NaN    NaN    37.0    NaN
Inh-CGE-EPHA4          NaN    NaN   107.0    NaN
Inh-CGE-FRAS1          NaN    NaN    83.0   27.0
Inh-CGE-MN1            NaN    NaN    44.0    NaN
Inh-CGE-OXR1           NaN    NaN    68.0   19.0
Inh-CGE-SOX13          NaN    NaN   148.0   32.0
Inh-CGE-TOX            NaN    NaN    99.0   17.0
Inh-MGE                7.0  468.0     NaN    NaN
Inh-MGE-ALCAM          NaN    NaN   121.0    NaN
Inh-MGE-CNTNAP2        NaN    NaN   389.0    NaN
Inh-MGE-ChC-UNC5B      NaN    NaN    87.0   30.0
Inh-MGE-ERBB4          NaN    NaN   217.0  164.0
Inh-MGE-MAN1A1         NaN    NaN   358.0  171.0
Inh-MGE-RBFOX1         NaN    NaN   117.0   86.0
Inh-eCGE            1211.0   51.0     NaN    NaN
Inh-eMGE            1101.0   55.0     NaN    NaN
MGC-1                 44.0   61.0   574.0  232.0
MGC-2                  NaN   12.0   172.0   60.0
ODC                    NaN    NaN  2152.0  231.0
OPC                   29.0  114.0   462.0  308.0
PC                    16.0    6.0   111.0   39.0
RG-1                3101.0    NaN     NaN    NaN
RG-2                 590.0    6.0     2.0    3.0
VLMC                   NaN   32.0   191.0   36.0

In [7]:
ct_ann.loc[ct_ann['region'] == 'CTX', ['L1 annotation', 'age_groups']].value_counts().to_frame().reset_index()\
    .pivot(columns='age_groups', index='L1 annotation')

count                   
age_groups       2T    3T adult infant
L1 annotation                         
Exc            3303  2176  4952   2328
Inh            2325  1004  2197    728
Non-Neuron      108   572  4797   1499
RG             3691     6     2      3

In [19]:
ct_ann.loc[(ct_ann['region'] == 'CTX') & (ct_ann['L2 annotation'] == 'Exc-UL'), 
           ['L3 annotation', 'age_groups']].value_counts().to_frame().reset_index()\
    .pivot(columns='age_groups', index='L3 annotation')

count                     
age_groups             2T     3T  adult infant
L3 annotation                                 
Exc-DL-1            639.0    NaN    NaN    NaN
Exc-DL-2            328.0    3.0    NaN    NaN
Exc-DL-ASTN1          2.0  122.0  166.0  102.0
Exc-DL-GRIK4          NaN    NaN  240.0  121.0
Exc-DL-GRIK4-TSHZ2    1.0  127.0    NaN    NaN
Exc-NP-TSHZ2          NaN    NaN   79.0   43.0

In [ ]:
ct_ann.loc[ct_ann['region'] == 'CTX'].groupby(['L3 annotation', 'Sample'])

In [25]:
# List cell types
print(*np.sort(ct_ann.loc[(ct_ann['region'] == 'CTX') & (ct_ann['age_groups'] == '2T'), 
                          'L3 annotation'].unique()), sep='\n')

Astro
EC
Exc-DL-1
Exc-DL-2
Exc-DL-ASTN1
Exc-DL-GRIK4-TSHZ2
Exc-UL-1
Inh-CGE
Inh-MGE
Inh-eCGE
Inh-eMGE
MGC-1
OPC
PC
RG-1
RG-2


In [4]:
# List brain regions
ct_ann['region'].unique()

array(['HPC', 'CTX'], dtype=object)

In [5]:
# List age groups
ct_ann['age_groups'].unique()

array(['3T', 'adult', '2T', 'infant'], dtype=object)

# Match sample names and filenames

In [12]:
avail_samples = [file for file in listdir("../../data/heffel/contacts/") 
                 if not file.endswith("tar.gz")]
avail_samples = [samp for samp in avail_samples if samp in ct_ann['Sample'].unique()]
print(f"Number of availables samples: {len(avail_samples)}") 

Number of availables samples: 33


# Get required cells

In [6]:
cts = ['RG-1']
reg = ['CTX']
age = ['2T']
ncells = 728

ids = ct_ann.loc[ct_ann['Sample'].isin(avail_samples) &
                 ct_ann['L1 annotation'].isin(cts) & 
                 ct_ann['region'].isin(reg) &
                 ct_ann['age_groups'].isin(age), ['Sample', 'Cell ID']]
# ids['sample_name'] = ids['Sample'].map(s2f)

# Sample cells
if (ncells != 0) and (ids.shape[0] > ncells):
    np.random.seed(0)
    rand_samp = np.random.choice(ids.index, size=ncells, replace=False)
    ids = ids.loc[rand_samp]

ids.loc[:, 'Cell ID'] = ids.loc[:, 'Cell ID'] + '_'
print("Using", ids.shape[0], "cells")
ids.head()

NameError: name 'avail_samples' is not defined

In [13]:
# Get cell filenames
import os


def find_cell_fname(id, cell_fnames):
    # cell_fname = cell_fnames[cell_fnames.str.startswith(id)]
    cell_fname = [fname for fname in cell_fnames if fname.startswith(id)]
    if len(cell_fname) == 0:
        print('Not found')
        return 'Not found'
    elif len(cell_fname) == 1:
        return cell_fname[0]
    else:
        print(cell_fname)
        return "More than 1 found"


def uniquify(path):
    """
    Make unique filepath by adding an integer suffix.
    Borrowed from: https://stackoverflow.com/questions/13852700/create-file-but-if-name-exists-add-number
    """
    filename, extension = os.path.splitext(path)
    counter = 1

    while os.path.exists(path):
        path = filename + "-" + str(counter) + extension
        counter += 1

    return path


def create_clr(reg, age, cts, clr_minres, chrom_sizes, ct_ann_path, contact_dir, 
               ncells=0, zoomify=True, zoomify_nproc=5, upload2higlass=False, 
               clr_exec_path='/opt/anaconda/bin/cooler'):

    # Read cell type annotation table
    ct_ann = pd.read_excel(ct_ann_path, skiprows=1)

    avail_samples = [file for file in listdir(contact_dir) 
                     if not file.endswith("tar.gz")]
    avail_samples = [samp for samp in avail_samples if samp in ct_ann['Sample'].unique()]
    print(f"Number of availables samples: {len(avail_samples)}") 

    ids = ct_ann.loc[
        (ct_ann['Sample'].isin(avail_samples)) &
        (ct_ann['L1 annotation'].isin(cts) | 
         ct_ann['L2 annotation'].isin(cts) | 
         ct_ann['L3 annotation'].isin(cts)) & 
        (ct_ann['region'] == reg) &
        (ct_ann['age_groups'] == age), 
        ['Sample', 'Cell ID', 'L1 annotation', 'L2 annotation', 'L3 annotation']
    ]
    # ids['sample_name'] = ids['Sample'].map(s2f)
    print(
        "Using cell types: ",
        "L1 annotation: " + ", ".join(ids['L1 annotation'].unique()),
        "L2 annotation: " + ", ".join(ids['L2 annotation'].unique()),
        "L3 annotation: " + ", ".join(ids['L3 annotation'].unique()),
        sep='\n'
    )
    avail_cells = ids.shape[0]
    
    # Sample cells
    if (ncells != 0) and (ids.shape[0] > ncells):
        np.random.seed(0)
        rand_samp = np.random.choice(ids.index, size=ncells, replace=False)
        ids = ids.loc[rand_samp]
    
    ids.loc[:, 'Cell ID'] = ids.loc[:, 'Cell ID'] + '_'
    print("Using", ids.shape[0], "out of", avail_cells, "cells")

    # cell_fnames
    for i, samp in enumerate(ids['Sample'].unique()):
        print(samp)
        # contact_dir = "/tank/projects/polycomb/data/heffel/contacts/"
        samp_dir = contact_dir + samp
        cell_fns = listdir(samp_dir)
        ids.loc[ids['Sample'] == samp, 'cell_fname'] = contact_dir + ids.loc[ids['Sample'] == samp, 'Sample'] + "/" + \
            ids.loc[ids['Sample'] == samp, 'Cell ID'].apply(find_cell_fname, cell_fnames=cell_fns)
    
    # Save to file
    cell_lst_fp = "tmp." + reg + '.' + '_'.join(cts) + '.' + age + \
                  f".{ids.shape[0]}_cells" + '.filelist.txt'
    if path.exists(cell_lst_fp):
        print("Filename for temporary storing list of filenames for contact files already exists. \
               Creating a unique filename..")
        cell_lst_fp = uniquify(cell_lst_fp)
    ids[['cell_fname']].to_csv(cell_lst_fp, header=False, index=False)
    
    # Concatenate ".3C.contact.tsv.gz" files into one
    # Might take several minutes
    pair_out_fn = reg + "." + '_'.join(cts) + '.' + age + \
                  f".{ids.shape[0]}_cells" + '.3C.contact.tsv.gz'
    if path.exists(pair_out_fn):
        print("Warning: filename for temporary storing merged contact files already exists. \
               Creating a unique filename..")
        pair_out_fn = uniquify(pair_out_fn)
    system(f"xargs cat < {cell_lst_fp} > {pair_out_fn}")
    system(f"rm {cell_lst_fp}")
    
    # CLI command to create a cooler file
    # chrom_sizes = '/tank/projects/tian2023/hg38.reduced.chrom.sizes'
    # clr_minres = 10_000
    # zoomify=True
    # zoomify_nproc = 2
    # upload2higlass =True
    
    clr_out_fn = reg + '.' + '_'.join(cts) + '.' + age + \
                 f".{ids.shape[0]}_cells" + f".{clr_minres // 1000}kb" + '.cool'
    system(f"{clr_exec_path} cload pairs --assembly hg38 -c1 1 -p1 2 -c2 3 -p2 4 \
           {chrom_sizes}:{int(clr_minres)} {pair_out_fn} {clr_out_fn}")
    if path.exists(clr_out_fn):
        system(f"rm {pair_out_fn}")
    
    # balance and zoomify cooler
    if zoomify:
        system(f"{clr_exec_path} zoomify -p {zoomify_nproc} -r {clr_minres}N --balance {clr_out_fn}")
    else:
        system(f"{clr_exec_path} balance {clr_out_fn}")
    
    # upload2higlass
    if upload2higlass:
        mcool_fn = re.sub(".cool$", ".mcool", clr_out_fn)
        system(f"cp {mcool_fn} /tank/hg-tmp/{mcool_fn}")
        system(
            f"docker exec higlass-container python higlass-server/manage.py ingest_tileset \
                --filename /tmp/{mcool_fn} \
                --filetype cooler \
                --datatype matrix \
                --coordSystem hg38 \
                --name {mcool_fn} \
                --project 'Heffel et al.'"
        )
        system(f"rm /tank/hg-tmp/{mcool_fn}")

    return

In [14]:
reg ='CTX'
cts = ['Exc', 'Inh']
clr_minres = 5_000
chrom_sizes = '/tank/projects/tian2023/hg38.reduced.chrom.sizes'
ct_ann_path = '/tank/projects/polycomb/data/heffel/supp_table_s3.cell_type_annotation.xlsx'
contact_dir = '/tank/projects/polycomb/data/heffel/contacts/'

for age in ['2T', '3T', 'infant', 'adult']:
    print(age)
    create_clr(reg, age, cts, clr_minres, chrom_sizes, ct_ann_path, contact_dir, ncells=3056,
               zoomify=False, zoomify_nproc=5, upload2higlass=False)

2T
Number of availables samples: 33
Using cell types: 
L1 annotation: Inh, Exc
L2 annotation: Inh-CGE, Exc-UL, Exc-DL, Inh-MGE
L3 annotation: Inh-eCGE, Exc-UL-1, Exc-DL-1, Inh-eMGE, Exc-DL-2, Inh-MGE, Inh-CGE, Exc-DL-ASTN1, Exc-DL-GRIK4-TSHZ2
Using 3056 out of 5628 cells
20210224_GW20b
20210316_GW20a
20210224_GW20a
20210224_GW23
20210224_GW18


  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.create:Writing chunk 0: /tank/projects/polycomb/results/2024.04.23/tmpe2igenbg.multi.cool::0
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.04.23/tmpe2igenbg.multi.cool::/0"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Writing chunk 1: /tank/projects/polycomb/results/2024.04.23/tmpe2igenbg.multi.cool::1
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.04.23/tmpe2igenbg.multi.cool::/1"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Writing chunk 2: /tank/projects/polycomb/results/2024.04.23/tmpe2igenbg.multi.cool::2
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.

3T
Number of availables samples: 33
Using cell types: 
L1 annotation: Inh, Exc
L2 annotation: Inh-CGE, Exc-DL, Exc-UL, Inh-MGE
L3 annotation: Inh-eCGE, Exc-DL-ASTN1, Inh-CGE, Exc-L1-3-CUX2, Exc-L5-6-PDZRN4, Exc-L4-RORB, Inh-MGE, Exc-UL-2, Inh-eMGE, Exc-DL-GRIK4-TSHZ2, Exc-UL-1, Exc-DL-2
Using 3056 out of 3180 cells
20210316_UA1822
20210316_UMB4267


  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.create:Writing chunk 0: /tank/projects/polycomb/results/2024.04.23/tmp_r9mhrkv.multi.cool::0
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.04.23/tmp_r9mhrkv.multi.cool::/0"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Writing chunk 1: /tank/projects/polycomb/results/2024.04.23/tmp_r9mhrkv.multi.cool::1
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.04.23/tmp_r9mhrkv.multi.cool::/1"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Writing chunk 2: /tank/projects/polycomb/results/2024.04.23/tmp_r9mhrkv.multi.cool::2
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.

infant
Number of availables samples: 33
Using cell types: 
L1 annotation: Inh, Exc
L2 annotation: Inh-CGE, Exc-UL, Exc-DL, Inh-MGE
L3 annotation: Inh-CGE-CHRNA2, Exc-L1-3-CUX2, Exc-L4-6-LRRK1, Exc-L4-RORB, Exc-DL-ASTN1, Exc-L5-6-PDZRN4, Exc-L1-3-NRXN2, Inh-MGE-ERBB4, Inh-MGE-RBFOX1, Inh-CGE-FRAS1, Inh-MGE-ChC-UNC5B, Exc-DL-GRIK4, Exc-L4-PLCH1, Inh-CGE-SOX13, Inh-CGE-OXR1, Inh-CGE-TOX, Exc-L4-5-FOXP2, Inh-MGE-MAN1A1, Exc-UL-1, Exc-NP-TSHZ2, Inh-CGE-ALK
Using 3056 out of 3056 cells
20191219_7mo
20210316_UA1922


  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.create:Writing chunk 0: /tank/projects/polycomb/results/2024.04.23/tmpw_s9ljc4.multi.cool::0
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.04.23/tmpw_s9ljc4.multi.cool::/0"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Writing chunk 1: /tank/projects/polycomb/results/2024.04.23/tmpw_s9ljc4.multi.cool::1
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.04.23/tmpw_s9ljc4.multi.cool::/1"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Writing chunk 2: /tank/projects/polycomb/results/2024.04.23/tmpw_s9ljc4.multi.cool::2
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.

adult
Number of availables samples: 33
Using cell types: 
L1 annotation: Exc, Inh
L2 annotation: Exc-UL, Exc-DL, Inh-MGE, Inh-CGE
L3 annotation: Exc-L4-5-FOXP2, Exc-L1-3-CUX2, Exc-DL-ASTN1, Exc-L4-RORB, Inh-MGE-MAN1A1, Inh-CGE-CLMP, Exc-L1-3-NRXN2, Exc-L5-6-PDZRN4, Inh-MGE-ALCAM, Exc-L4-6-LRRK1, Inh-CGE-SOX13, Inh-MGE-RBFOX1, Exc-DL-GRIK4, Inh-CGE-MN1, Inh-CGE-CHRNA2, Inh-MGE-ERBB4, Inh-MGE-CNTNAP2, Exc-L4-PLCH1, Inh-CGE-TOX, Exc-L4-5-TOX, Exc-NP-TSHZ2, Inh-MGE-ChC-UNC5B, Inh-CGE-EPHA4, Inh-CGE-ALK, Inh-CGE-OXR1, Inh-CGE-FRAS1
Using 3056 out of 5811 cells
20210528_UMB5621
20210111_UMB1863
20210505_UMB6096
20210224_UMB5577
20210528_UMB5577
20210505_UMB5577


  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.create:Writing chunk 0: /tank/projects/polycomb/results/2024.04.23/tmp2hi_e0w0.multi.cool::0
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.04.23/tmp2hi_e0w0.multi.cool::/0"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Writing chunk 1: /tank/projects/polycomb/results/2024.04.23/tmp2hi_e0w0.multi.cool::1
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.04.23/tmp2hi_e0w0.multi.cool::/1"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Writing chunk 2: /tank/projects/polycomb/results/2024.04.23/tmp2hi_e0w0.multi.cool::2
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.

In [23]:
reg ='CTX'
cts = ['RG-1']
clr_minres = 10_000
chrom_sizes = '/tank/projects/tian2023/hg38.reduced.chrom.sizes'
ct_ann_path = '/tank/projects/polycomb/data/heffel/supp_table_s3.cell_type_annotation.xlsx'
contact_dir = '/tank/projects/polycomb/data/heffel/contacts/'

for age in ['2T']:
    print(age)
    create_clr(reg, age, cts, clr_minres, chrom_sizes, ct_ann_path, contact_dir, ncells=3056,
               zoomify=True, zoomify_nproc=5, upload2higlass=True)

2T
Number of availables samples: 33
Using cell types: 
L1 annotation: RG
L2 annotation: RG
L3 annotation: RG-1
Using 3056 cells
20210316_GW20a
20210224_GW18
20210224_GW20b
20210224_GW20a
20210224_GW23
20210316_UMB4267
20210316_UA1822


  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.create:Writing chunk 0: /tank/projects/polycomb/results/2024.04.23/tmp87q7wxg5.multi.cool::0
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.04.23/tmp87q7wxg5.multi.cool::/0"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Writing chunk 1: /tank/projects/polycomb/results/2024.04.23/tmp87q7wxg5.multi.cool::1
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.04.23/tmp87q7wxg5.multi.cool::/1"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Writing chunk 2: /tank/projects/polycomb/results/2024.04.23/tmp87q7wxg5.multi.cool::2
INFO:cooler.create:Creating cooler at "/tank/projects/polycomb/results/2024.

In [15]:
!pwd

/tank/projects/polycomb/results/2024.04.23


# Script to create bulk contact maps from Heffel et al.

In [7]:
%%writefile make_clr_heffel.py

import pandas as pd
import cooler
import matplotlib.pyplot as plt
import numpy as np
from os import listdir, system, path
import re
import argparse

# Set argparse
parser = argparse.ArgumentParser(description="Make cooler matrices from single-cell contact pair files.")

parser.add_argument('-r', '--reg', help='Brain region: "CTX" (cortex) or "HPC" (hippocampus)', 
                    type=str, required=True)
parser.add_argument('-c', '--cts', help='Comma-delimited list of cell-types', 
                    type=str, required=True)
parser.add_argument('-a', '--age', help='Sample age: "2T", "3T", "infant", or "adult"', 
                    type=str, required=True)
parser.add_argument('-bs', '--binsize', help='Smallest binsize for the resulting cooler', 
                    type=int, required=True)
parser.add_argument('-n', '--ncells', help='Limit number of used cells to <ncells> sampled randomly', 
                    type=int, default=0)
parser.add_argument('-s', '--samples', help='Comma-delimited list of samples', 
                    type=str, default='all')
parser.add_argument('-ls', '--list-samples', help='List available samples and exit', 
                    default=False, action='store_true')
parser.add_argument('-ch', '--chunksize', help='chunksize parameter for cooler cload pairs function', 
                    type=int, default=None)
parser.add_argument('-hg', '--higlass', 
                    help='Whether to upload map to higlass in "Tian et al. 2023" project', 
                    default=False, action='store_true')
args = parser.parse_args()


chrom_sizes = '/tank/projects/tian2023/hg38.reduced.chrom.sizes'
ct_ann_path = '/tank/projects/polycomb/data/heffel/supp_table_s3.cell_type_annotation.xlsx'
contact_dir = '/tank/projects/polycomb/data/heffel/contacts/'


def find_cell_fname(id, cell_fnames):
    # cell_fname = cell_fnames[cell_fnames.str.startswith(id)]
    cell_fname = [fname for fname in cell_fnames if fname.startswith(id)]
    if len(cell_fname) == 0:
        print('Not found')
        return 'Not found'
    elif len(cell_fname) == 1:
        return cell_fname[0]
    else:
        print(cell_fname)
        return "More than 1 found"


def uniquify(path):
    """
    Make unique filepath by adding an integer suffix.
    Borrowed from: https://stackoverflow.com/questions/13852700/create-file-but-if-name-exists-add-number
    """
    filename, extension = path.splitext(path)
    counter = 1

    while path.exists(path):
        path = filename + "-" + str(counter) + extension
        counter += 1

    return path


def create_clr(
        reg, 
        age, 
        cts, 
        clr_minres, 
        chrom_sizes, 
        ct_ann_path, 
        contact_dir, 
        ncells=0, 
        use_samps=None,
        chunksize=None,
        list_samples=False,
        zoomify=True, 
        zoomify_nproc=5, 
        upload2higlass=False, 
        clr_exec_path='/opt/anaconda/bin/cooler'
    ):

    # Read cell type annotation table
    ct_ann = pd.read_excel(ct_ann_path, skiprows=1)

    avail_samples = [file for file in listdir(contact_dir) 
                     if not file.endswith("tar.gz")]
    avail_samples = [samp for samp in avail_samples if samp in ct_ann['Sample'].unique()]
    print(f"Number of availables samples: {len(avail_samples)}") 

    ids = ct_ann.loc[(ct_ann['Sample'].isin(avail_samples)) &
                     (ct_ann['L1 annotation'].isin(cts) | 
                      ct_ann['L2 annotation'].isin(cts) |
                      ct_ann['L3 annotation'].isin(cts) |
                      ct_ann['L4 annotation'].isin(cts)) & 
                     (ct_ann['region'] == reg) &
                     (ct_ann['age_groups'] == age), ['Sample', 'Cell ID']]
    
    if list_samples:
        print("Samples for required parameters: ", ids['Sample'].unique())
        return

    if use_samps != ['all']:
        ids = ids.loc[ids['Sample'].isin(use_samps)]

    print("Using samples: ", ids['Sample'].unique())
    
    # Sample cells
    if (ncells != 0) and (ids.shape[0] > ncells):
        np.random.seed(0)
        rand_samp = np.random.choice(ids.index, size=ncells, replace=False)
        ids = ids.loc[rand_samp]
    
    ids.loc[:, 'Cell ID'] = ids.loc[:, 'Cell ID'] + '_'
    print("Using", ids.shape[0], "cells")

    # cell_fnames
    for i, samp in enumerate(ids['Sample'].unique()):
        print(samp)
        # contact_dir = "/tank/projects/polycomb/data/heffel/contacts/"
        samp_dir = contact_dir + samp
        cell_fns = listdir(samp_dir)
        ids.loc[ids['Sample'] == samp, 'cell_fname'] = contact_dir + ids.loc[ids['Sample'] == samp, 'Sample'] + "/" + \
            ids.loc[ids['Sample'] == samp, 'Cell ID'].apply(find_cell_fname, cell_fnames=cell_fns)
    
    # Save to file
    cell_lst_fp = "tmp." + reg + '.' + '_'.join(cts) + '.' + age + \
                  f".{ids.shape[0]}_cells" + '.' + '_'.join(use_samps) + '.filelist.txt'
    if path.exists(cell_lst_fp):
        print("Filename for temporary storing list of filenames for contact files already exists. \
               Creating a unique filename..")
        cell_lst_fp = uniquify(cell_lst_fp)
    ids[['cell_fname']].to_csv(cell_lst_fp, header=False, index=False)
    
    # Concatenate ".3C.contact.tsv.gz" files into one
    # Might take several minutes
    pair_out_fn = reg + "." + '_'.join(cts) + '.' + age + \
                  f".{ids.shape[0]}_cells" + '.' + '_'.join(use_samps) + '.3C.contact.tsv.gz'
    if path.exists(pair_out_fn):
        print("Warning: filename for temporary storing merged contact files already exists. \
               Creating a unique filename..")
        pair_out_fn = uniquify(pair_out_fn)
    system(f"xargs cat < {cell_lst_fp} > {pair_out_fn}")
    system(f"rm {cell_lst_fp}")
    
    # CLI command to create a cooler file   
    clr_out_fn = reg + '.' + '_'.join(cts) + '.' + age + \
                 f".{ids.shape[0]}_cells" + '.' + '_'.join(use_samps) + f".{clr_minres // 1000}kb" + '.cool'
    if chunksize is not None:
        system(f"{clr_exec_path} cload pairs --assembly hg38 --chunksize {chunksize} \
               -c1 1 -p1 2 -c2 3 -p2 4 \
               {chrom_sizes}:{int(clr_minres)} {pair_out_fn} {clr_out_fn}")
    else:
        system(f"{clr_exec_path} cload pairs --assembly hg38 -c1 1 -p1 2 -c2 3 -p2 4 \
               {chrom_sizes}:{int(clr_minres)} {pair_out_fn} {clr_out_fn}")
    if path.exists(clr_out_fn):
        system(f"rm {pair_out_fn}")
    
    # balance and zoomify cooler
    if zoomify:
        system(f"{clr_exec_path} zoomify -p {zoomify_nproc} -r {clr_minres}N --balance {clr_out_fn}")
    else:
        system(f"{clr_exec_path} balance {clr_out_fn}")
    
    # upload2higlass
    if upload2higlass:
        mcool_fn = re.sub(".cool$", ".mcool", clr_out_fn)
        system(f"cp {mcool_fn} /tank/hg-tmp/{mcool_fn}")
        system(
            f"docker exec higlass-container python higlass-server/manage.py ingest_tileset \
                --filename /tmp/{mcool_fn} \
                --filetype cooler \
                --datatype matrix \
                --coordSystem hg38 \
                --name {mcool_fn} \
                --project 'Heffel et al.'"
        )
        system(f"rm /tank/hg-tmp/{mcool_fn}")

    return


## MAIN
create_clr(
    args.reg, 
    args.age, 
    args.cts.split(','), 
    args.binsize, 
    chrom_sizes, 
    ct_ann_path, 
    contact_dir, 
    ncells=args.ncells,
    use_samps=args.samples.split(','),
    chunksize=args.chunksize,
    list_samples=args.list_samples,
    zoomify=True, 
    zoomify_nproc=5, 
    upload2higlass=args.higlass
)

Overwriting make_clr_heffel.py


# Prepare data for 3C trajectories

In [5]:
# Read cell type annotation
ct_ann_path = '/tank/projects/polycomb/data/heffel/supp_table_s3.cell_type_annotation.xlsx'
ct_ann = pd.read_excel(ct_ann_path, skiprows=1)
ct_ann.head()

,Cell ID,L1 annotation,L2 annotation,L3 annotation,L4 annotation,mCCC/CCC,mCH/CH,mCG/CG,Sample,region,age_groups,TotalInt
0,210830-4267-HPC-3-C07_F15,Non-Neuron,Glial-Non-Neuron,OPC,3T_OPC,0.00859,0.01151,0.75400,210830-4267-HPC,HPC,3T,138582
1,210830-4267-HPC-3-C07_C2,Exc,Exc-CA,Exc-CA-TSHZ2-MEIS2,3T_Exc-CA-TSHZ2-MEIS2,0.00927,0.01860,0.75798,210830-4267-HPC,HPC,3T,139219
2,210830-4267-HPC-3-C07_J11,Exc,Exc-ENT,Exc-ENT,NaN,0.00780,0.01168,0.73789,210830-4267-HPC,HPC,3T,194860
3,210830-4267-HPC-3-C07_C24,Non-Neuron,Glial-Non-Neuron,OPC,3T_OPC,0.00815,0.01218,0.72286,210830-4267-HPC,HPC,3T,137466
4,210830-4267-HPC-3-C07_J7,Exc,Exc-CA,Exc-CA1,3T_Exc-CA1,0.00903,0.02066,0.74035,210830-4267-HPC,HPC,3T,203616


In [6]:
# List cell types
ct_ann['L2 annotation'].unique()

array(['Glial-Non-Neuron', 'Exc-CA', 'Exc-ENT', 'Inh-MGE', 'Inh-CGE',
       'RG', 'Non-Neuron', 'Exc-DG', 'Exc-UL', 'Exc-DL'], dtype=object)

In [7]:
# List available samples
contact_dir = '/tank/projects/polycomb/data/heffel/contacts/'
avail_samples = [file for file in listdir(contact_dir) 
                 if not file.endswith("tar.gz")]
avail_samples = [samp for samp in avail_samples if samp in ct_ann['Sample'].unique()]

In [8]:
ct_inh = ct_ann.loc[(ct_ann['L2 annotation'].isin(['RG', 'Inh-MGE', 'Inh-CGE'])) & 
                    (ct_ann['region'] == 'CTX') & 
                    (ct_ann['Sample'].isin(avail_samples))]
# ct_inh.loc[:, 'Cell ID'] = ct_inh.loc[:, 'Cell ID'] + '_'
ct_inh

,Cell ID,L1 annotation,L2 annotation,L3 annotation,L4 annotation,mCCC/CCC,mCH/CH,mCG/CG,Sample,region,age_groups,TotalInt
23372,2-1-D2-P2,Inh,Inh-CGE,Inh-CGE-CHRNA2,infant_Inh-CGE-CHRNA2,0.00722,0.02629,0.78827,20191219_7mo,CTX,infant,120734
23385,2-3-D6-G6,Inh,Inh-MGE,Inh-MGE-ERBB4,infant_Inh-MGE-ERBB4,0.00824,0.03775,0.83090,20191219_7mo,CTX,infant,173379
23389,1-5-B10-C9,Inh,Inh-MGE,Inh-MGE-RBFOX1,infant_Inh-MGE-RBFOX1,0.00496,0.01639,0.78147,20191219_7mo,CTX,infant,179482
23397,1-4-B8-K7,Inh,Inh-CGE,Inh-CGE-CHRNA2,infant_Inh-CGE-CHRNA2,0.00643,0.02342,0.80009,20191219_7mo,CTX,infant,159747
23403,1-4-B8-D7,Inh,Inh-CGE,Inh-CGE-FRAS1,infant_Inh-CGE-FRAS1,0.01096,0.03116,0.80210,20191219_7mo,CTX,infant,178821
...,...,...,...,...,...,...,...,...,...,...,...,...
49709,20210528-UMB5621-preAb-PFC-B2_N19,Inh,Inh-CGE,Inh-CGE-CHRNA2,adult_Inh-CGE-CHRNA2,0.01819,0.08389,0.82524,20210528_UMB5621,CTX,adult,289849
49718,20210528-UMB5621-preAb-PFC-B2_O8,Inh,Inh-MGE,Inh-MGE-ALCAM,adult_Inh-MGE-ALCAM,0.01647,0.07608,0.81029,20210528_UMB5621,CTX,adult,217473
49722,20210528-UMB5621-preAb-PFC-B2_N16,Inh,Inh-MGE,Inh-MGE-CNTNAP2,adult_Inh-MGE-CNTNAP2,0.01002,0.04783,0.81749,20210528_UMB5621,CTX,adult,279427
49734,20210528-UMB5621-preAb-PFC-B2_N18,Inh,Inh-MGE,Inh-MGE-ERBB4,adult_Inh-MGE-ERBB4,0.01376,0.06803,0.82184,20210528_UMB5621,CTX,adult,377600


In [9]:
# cell_fnames
def find_cell_fname(id, cell_fnames):
    # cell_fname = cell_fnames[cell_fnames.str.startswith(id)]
    id += '_'
    cell_fname = [fname for fname in cell_fnames if fname.startswith(id)]
    if len(cell_fname) == 0:
        print('Not found')
        return 'Not found'
    elif len(cell_fname) == 1:
        return cell_fname[0]
    else:
        print(cell_fname)
        return "More than 1 found"

clr_dir = '/tank/projects/polycomb/data/heffel/cools/'
for i, samp in enumerate(ct_inh['Sample'].unique()):
    print(samp)
    # contact_dir = "/tank/projects/polycomb/data/heffel/contacts/"
    samp_dir = contact_dir + samp
    cell_fns = listdir(samp_dir)
    ct_inh.loc[ct_inh['Sample'] == samp, 'contacts_fpath'] = samp_dir + "/" + \
        ct_inh.loc[ct_inh['Sample'] == samp, 'Cell ID'].apply(find_cell_fname, cell_fnames=cell_fns)

    ct_inh.loc[ct_inh['Sample'] == samp, 'cooler_fname'] = \
        ct_inh.loc[ct_inh['Sample'] == samp, 'Cell ID'] + '.10kb.cool'
    ct_inh.loc[ct_inh['Sample'] == samp, 'cooler_fpath'] = \
        clr_dir + ct_inh.loc[ct_inh['Sample'] == samp, 'cooler_fname']

ct_inh

20191219_7mo


/tmp/ipykernel_1834470/1349532756.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ct_inh.loc[ct_inh['Sample'] == samp, 'contacts_fpath'] = samp_dir + "/" + \
/tmp/ipykernel_1834470/1349532756.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ct_inh.loc[ct_inh['Sample'] == samp, 'cooler_fname'] = \
/tmp/ipykernel_1834470/1349532756.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

20210111_UMB1863
20210224_GW18
20210224_GW20a
20210224_GW20b
20210224_GW23
20210224_UMB5577
20210316_GW20a
20210316_UA1822
20210316_UA1922
20210316_UMB4267
20210505_UMB5577
20210505_UMB6096
20210528_UMB5577
20210528_UMB5621


,Cell ID,L1 annotation,L2 annotation,L3 annotation,L4 annotation,mCCC/CCC,mCH/CH,mCG/CG,Sample,region,age_groups,TotalInt,contacts_fpath,cooler_fname,cooler_fpath
23372,2-1-D2-P2,Inh,Inh-CGE,Inh-CGE-CHRNA2,infant_Inh-CGE-CHRNA2,0.00722,0.02629,0.78827,20191219_7mo,CTX,infant,120734,/tank/projects/polycomb/data/heffel/contacts/2...,2-1-D2-P2.10kb.cool,/tank/projects/polycomb/data/heffel/cools/2-1-...
23385,2-3-D6-G6,Inh,Inh-MGE,Inh-MGE-ERBB4,infant_Inh-MGE-ERBB4,0.00824,0.03775,0.83090,20191219_7mo,CTX,infant,173379,/tank/projects/polycomb/data/heffel/contacts/2...,2-3-D6-G6.10kb.cool,/tank/projects/polycomb/data/heffel/cools/2-3-...
23389,1-5-B10-C9,Inh,Inh-MGE,Inh-MGE-RBFOX1,infant_Inh-MGE-RBFOX1,0.00496,0.01639,0.78147,20191219_7mo,CTX,infant,179482,/tank/projects/polycomb/data/heffel/contacts/2...,1-5-B10-C9.10kb.cool,/tank/projects/polycomb/data/heffel/cools/1-5-...
23397,1-4-B8-K7,Inh,Inh-CGE,Inh-CGE-CHRNA2,infant_Inh-CGE-CHRNA2,0.00643,0.02342,0.80009,20191219_7mo,CTX,infant,159747,/tank/projects/polycomb/data/heffel/contacts/2...,1-4-B8-K7.10kb.cool,/tank/projects/polycomb/data/heffel/cools/1-4-...
23403,1-4-B8-D7,Inh,Inh-CGE,Inh-CGE-FRAS1,infant_Inh-CGE-FRAS1,0.01096,0.03116,0.80210,20191219_7mo,CTX,infant,178821,/tank/projects/polycomb/data/heffel/contacts/2...,1-4-B8-D7.10kb.cool,/tank/projects/polycomb/data/heffel/cools/1-4-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49709,20210528-UMB5621-preAb-PFC-B2_N19,Inh,Inh-CGE,Inh-CGE-CHRNA2,adult_Inh-CGE-CHRNA2,0.01819,0.08389,0.82524,20210528_UMB5621,CTX,adult,289849,/tank/projects/polycomb/data/heffel/contacts/2...,20210528-UMB5621-preAb-PFC-B2_N19.10kb.cool,/tank/projects/polycomb/data/heffel/cools/2021...
49718,20210528-UMB5621-preAb-PFC-B2_O8,Inh,Inh-MGE,Inh-MGE-ALCAM,adult_Inh-MGE-ALCAM,0.01647,0.07608,0.81029,20210528_UMB5621,CTX,adult,217473,/tank/projects/polycomb/data/heffel/contacts/2...,20210528-UMB5621-preAb-PFC-B2_O8.10kb.cool,/tank/projects/polycomb/data/heffel/cools/2021...
49722,20210528-UMB5621-preAb-PFC-B2_N16,Inh,Inh-MGE,Inh-MGE-CNTNAP2,adult_Inh-MGE-CNTNAP2,0.01002,0.04783,0.81749,20210528_UMB5621,CTX,adult,279427,/tank/projects/polycomb/data/heffel/contacts/2...,20210528-UMB5621-preAb-PFC-B2_N16.10kb.cool,/tank/projects/polycomb/data/heffel/cools/2021...
49734,20210528-UMB5621-preAb-PFC-B2_N18,Inh,Inh-MGE,Inh-MGE-ERBB4,adult_Inh-MGE-ERBB4,0.01376,0.06803,0.82184,20210528_UMB5621,CTX,adult,377600,/tank/projects/polycomb/data/heffel/contacts/2...,20210528-UMB5621-preAb-PFC-B2_N18.10kb.cool,/tank/projects/polycomb/data/heffel/cools/2021...


In [10]:
ct_inh[['contacts_fpath']].to_csv("contact_fpaths.csv", header=False, index=False)

In [63]:
ct_inh.to_csv("metadata.heffel.tsv.gz", index=False, header=True, sep='\t')

In [58]:
ct_inh.shape

(9412, 15)

In [56]:
# Missing interneurons 
exist_clrs = listdir(clr_dir)
s_miss = ct_inh.loc[~ct_inh['cooler_fname'].isin(exist_clrs)].reset_index(drop=True)
s_miss.head()

,Cell ID,L1 annotation,L2 annotation,L3 annotation,L4 annotation,mCCC/CCC,mCH/CH,mCG/CG,Sample,region,age_groups,TotalInt,cont_fpath,clr_fname,clr_fpath


In [53]:
# save to file
s_miss[['contacts_fpath', 'cooler_fpath']].to_csv("missing_coolers.heffel.txt", sep=' ', index=False, header=False)

In [57]:
s_miss.shape

(0, 15)

In [34]:
!pwd

/tank/projects/polycomb/results/2024.04.23


In [35]:
s_miss.loc[0, 'cont_fname']

'/tank/projects/polycomb/data/heffel/contacts/20191219_7mo/2-1-D2-P2_all_merged_3split_filtered.bam_multi_split_aligned.txt_no_less_2_contacts.txt.gz'

In [46]:
%%writefile make_missing_clrs_heffel.sh
CHROM_SIZES="/tank/projects/tian2023/hg38.reduced.chrom.sizes"
MISS_LIST="/tank/projects/polycomb/results/2024.04.23/missing_coolers.heffel.txt"

COUNT=0
while IFS=" " read -r CONTACT_FPATH CLR_FPATH REMAINDER
do
    echo ${CONTACT_FPATH}
    echo ${CLR_FPATH}
    echo ${COUNT}
    cooler cload pairs --assembly hg38 -c1 1 -p1 2 -c2 3 -p2 4 "${CHROM_SIZES}:10000" \
        --chunksize 150000000 \
        "${CONTACT_FPATH}" \
        "${CLR_FPATH}"
    COUNT=${COUNT}+1
done < "${MISS_LIST}"

Overwriting make_missing_clrs_heffel.sh


In [50]:
clr = cooler.Cooler("/tank/projects/polycomb/data/heffel/cools/2-1-D2-P2.10kb.cool")
clr.pixels()[:]['count'].sum()

120734

# Create remaining coolers for polycomb paper

In [2]:
# Read cell type annotation
ct_ann_path = '/tank/projects/polycomb/data/heffel/supp_table_s3.cell_type_annotation.xlsx'
ct_ann = pd.read_excel(ct_ann_path, skiprows=1)
ct_ann.head()

,Cell ID,L1 annotation,L2 annotation,L3 annotation,L4 annotation,mCCC/CCC,mCH/CH,mCG/CG,Sample,region,age_groups,TotalInt
0,210830-4267-HPC-3-C07_F15,Non-Neuron,Glial-Non-Neuron,OPC,3T_OPC,0.00859,0.01151,0.75400,210830-4267-HPC,HPC,3T,138582
1,210830-4267-HPC-3-C07_C2,Exc,Exc-CA,Exc-CA-TSHZ2-MEIS2,3T_Exc-CA-TSHZ2-MEIS2,0.00927,0.01860,0.75798,210830-4267-HPC,HPC,3T,139219
2,210830-4267-HPC-3-C07_J11,Exc,Exc-ENT,Exc-ENT,NaN,0.00780,0.01168,0.73789,210830-4267-HPC,HPC,3T,194860
3,210830-4267-HPC-3-C07_C24,Non-Neuron,Glial-Non-Neuron,OPC,3T_OPC,0.00815,0.01218,0.72286,210830-4267-HPC,HPC,3T,137466
4,210830-4267-HPC-3-C07_J7,Exc,Exc-CA,Exc-CA1,3T_Exc-CA1,0.00903,0.02066,0.74035,210830-4267-HPC,HPC,3T,203616


In [17]:
# List cell types
ct_ann['L2 annotation'].unique()

array(['Glial-Non-Neuron', 'Exc-CA', 'Exc-ENT', 'Inh-MGE', 'Inh-CGE',
       'RG', 'Non-Neuron', 'Exc-DG', 'Exc-UL', 'Exc-DL'], dtype=object)

In [4]:
# List available samples
contact_dir = '/tank/projects/polycomb/data/heffel/contacts/'
avail_samples = [file for file in listdir(contact_dir) 
                 if not file.endswith("tar.gz")]
avail_samples = [samp for samp in avail_samples if samp in ct_ann['Sample'].unique()]

In [5]:
ct_ctx = ct_ann.loc[(ct_ann['region'] == 'CTX') & 
                    (ct_ann['Sample'].isin(avail_samples))]
ct_ctx

,Cell ID,L1 annotation,L2 annotation,L3 annotation,L4 annotation,mCCC/CCC,mCH/CH,mCG/CG,Sample,region,age_groups,TotalInt
23372,2-1-D2-P2,Inh,Inh-CGE,Inh-CGE-CHRNA2,infant_Inh-CGE-CHRNA2,0.00722,0.02629,0.78827,20191219_7mo,CTX,infant,120734
23373,1-1-B2-L2,Exc,Exc-UL,Exc-L1-3-CUX2,infant_Exc-L1-3-CUX2,0.00737,0.02666,0.76238,20191219_7mo,CTX,infant,136162
23374,1-4-B8-J20,Non-Neuron,Glial-Non-Neuron,OPC,infant_OPC,0.00506,0.00713,0.76099,20191219_7mo,CTX,infant,109104
23375,1-6-B12-O12,Exc,Exc-UL,Exc-L4-6-LRRK1,infant_Exc-L4-6-LRRK1,0.00606,0.01943,0.75997,20191219_7mo,CTX,infant,151320
23376,1-4-B8-D20,Non-Neuron,Non-Neuron,EC,infant_EC,0.00464,0.00696,0.71418,20191219_7mo,CTX,infant,103255
...,...,...,...,...,...,...,...,...,...,...,...,...
49737,20210528-UMB5621-preAb-PFC-B2_O22,Exc,Exc-UL,Exc-L1-3-CUX2,adult_Exc-L1-3-CUX2,0.00964,0.04201,0.75407,20210528_UMB5621,CTX,adult,295484
49738,20210528-UMB5621-preAb-PFC-B2_P3,Exc,Exc-UL,Exc-L1-3-CUX2,adult_Exc-L1-3-CUX2,0.00871,0.04098,0.75080,20210528_UMB5621,CTX,adult,199484
49739,20210528-UMB5621-preAb-PFC-B2_P13,Exc,Exc-UL,Exc-L4-5-TOX,adult_Exc-L4-5-TOX,0.01463,0.07267,0.79455,20210528_UMB5621,CTX,adult,276870
49740,20210528-UMB5621-preAb-PFC-B2_P14,Inh,Inh-CGE,Inh-CGE-CHRNA2,adult_Inh-CGE-CHRNA2,0.01027,0.04177,0.80875,20210528_UMB5621,CTX,adult,315830


In [7]:
# cell_fnames
def find_cell_fname(id, cell_fnames):
    # cell_fname = cell_fnames[cell_fnames.str.startswith(id)]
    id += '_'
    cell_fname = [fname for fname in cell_fnames if fname.startswith(id)]
    if len(cell_fname) == 0:
        print('Not found')
        return 'Not found'
    elif len(cell_fname) == 1:
        return cell_fname[0]
    else:
        print(cell_fname)
        return "More than 1 found"

clr_dir = '/tank/projects/polycomb/data/heffel/cools/'
for i, samp in enumerate(ct_ctx['Sample'].unique()):
    print(samp)
    # contact_dir = "/tank/projects/polycomb/data/heffel/contacts/"
    samp_dir = contact_dir + samp
    cell_fns = listdir(samp_dir)
    ct_ctx.loc[ct_ctx['Sample'] == samp, 'contacts_fpath'] = samp_dir + "/" + \
        ct_ctx.loc[ct_ctx['Sample'] == samp, 'Cell ID'].apply(find_cell_fname, cell_fnames=cell_fns)

    ct_ctx.loc[ct_ctx['Sample'] == samp, 'cooler_fname'] = \
        ct_ctx.loc[ct_ctx['Sample'] == samp, 'Cell ID'] + '.10kb.cool'
    ct_ctx.loc[ct_ctx['Sample'] == samp, 'cooler_fpath'] = \
        clr_dir + ct_ctx.loc[ct_ctx['Sample'] == samp, 'cooler_fname']

ct_ctx

20191219_7mo
20210111_UMB1863
20210224_GW18
20210224_GW20a
20210224_GW20b
20210224_GW23
20210224_UMB5577
20210316_GW20a
20210316_UA1822
20210316_UA1922
20210316_UMB4267
20210505_UMB5577
20210505_UMB6096
20210528_UMB5577
20210528_UMB5621


,Cell ID,L1 annotation,L2 annotation,L3 annotation,L4 annotation,mCCC/CCC,mCH/CH,mCG/CG,Sample,region,age_groups,TotalInt,contacts_fpath,cooler_fname,cooler_fpath
23372,2-1-D2-P2,Inh,Inh-CGE,Inh-CGE-CHRNA2,infant_Inh-CGE-CHRNA2,0.00722,0.02629,0.78827,20191219_7mo,CTX,infant,120734,/tank/projects/polycomb/data/heffel/contacts/2...,2-1-D2-P2.10kb.cool,/tank/projects/polycomb/data/heffel/cools/2-1-...
23373,1-1-B2-L2,Exc,Exc-UL,Exc-L1-3-CUX2,infant_Exc-L1-3-CUX2,0.00737,0.02666,0.76238,20191219_7mo,CTX,infant,136162,/tank/projects/polycomb/data/heffel/contacts/2...,1-1-B2-L2.10kb.cool,/tank/projects/polycomb/data/heffel/cools/1-1-...
23374,1-4-B8-J20,Non-Neuron,Glial-Non-Neuron,OPC,infant_OPC,0.00506,0.00713,0.76099,20191219_7mo,CTX,infant,109104,/tank/projects/polycomb/data/heffel/contacts/2...,1-4-B8-J20.10kb.cool,/tank/projects/polycomb/data/heffel/cools/1-4-...
23375,1-6-B12-O12,Exc,Exc-UL,Exc-L4-6-LRRK1,infant_Exc-L4-6-LRRK1,0.00606,0.01943,0.75997,20191219_7mo,CTX,infant,151320,/tank/projects/polycomb/data/heffel/contacts/2...,1-6-B12-O12.10kb.cool,/tank/projects/polycomb/data/heffel/cools/1-6-...
23376,1-4-B8-D20,Non-Neuron,Non-Neuron,EC,infant_EC,0.00464,0.00696,0.71418,20191219_7mo,CTX,infant,103255,/tank/projects/polycomb/data/heffel/contacts/2...,1-4-B8-D20.10kb.cool,/tank/projects/polycomb/data/heffel/cools/1-4-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49737,20210528-UMB5621-preAb-PFC-B2_O22,Exc,Exc-UL,Exc-L1-3-CUX2,adult_Exc-L1-3-CUX2,0.00964,0.04201,0.75407,20210528_UMB5621,CTX,adult,295484,/tank/projects/polycomb/data/heffel/contacts/2...,20210528-UMB5621-preAb-PFC-B2_O22.10kb.cool,/tank/projects/polycomb/data/heffel/cools/2021...
49738,20210528-UMB5621-preAb-PFC-B2_P3,Exc,Exc-UL,Exc-L1-3-CUX2,adult_Exc-L1-3-CUX2,0.00871,0.04098,0.75080,20210528_UMB5621,CTX,adult,199484,/tank/projects/polycomb/data/heffel/contacts/2...,20210528-UMB5621-preAb-PFC-B2_P3.10kb.cool,/tank/projects/polycomb/data/heffel/cools/2021...
49739,20210528-UMB5621-preAb-PFC-B2_P13,Exc,Exc-UL,Exc-L4-5-TOX,adult_Exc-L4-5-TOX,0.01463,0.07267,0.79455,20210528_UMB5621,CTX,adult,276870,/tank/projects/polycomb/data/heffel/contacts/2...,20210528-UMB5621-preAb-PFC-B2_P13.10kb.cool,/tank/projects/polycomb/data/heffel/cools/2021...
49740,20210528-UMB5621-preAb-PFC-B2_P14,Inh,Inh-CGE,Inh-CGE-CHRNA2,adult_Inh-CGE-CHRNA2,0.01027,0.04177,0.80875,20210528_UMB5621,CTX,adult,315830,/tank/projects/polycomb/data/heffel/contacts/2...,20210528-UMB5621-preAb-PFC-B2_P14.10kb.cool,/tank/projects/polycomb/data/heffel/cools/2021...


In [19]:
!pwd

/tank/projects/polycomb/results/2024.04.23


In [8]:
ct_ctx.to_csv("metadata.heffel.all_ctx.tsv.gz", index=False, header=True, sep='\t')

In [58]:
ct_ctx.shape

(9412, 15)

In [18]:
# Missing cells 
exist_clrs = listdir(clr_dir)
s_miss = ct_ctx.loc[~ct_ctx['cooler_fname'].isin(exist_clrs)].reset_index(drop=True)
s_miss.head()

,Cell ID,L1 annotation,L2 annotation,L3 annotation,L4 annotation,mCCC/CCC,mCH/CH,mCG/CG,Sample,region,age_groups,TotalInt,contacts_fpath,cooler_fname,cooler_fpath


In [10]:
# save to file
s_miss[['contacts_fpath', 'cooler_fpath']].to_csv("missing_coolers.heffel.txt", sep=' ', index=False, header=False)

In [15]:
s_miss.shape

(16892, 15)

In [12]:
!pwd

/tank/projects/polycomb/results/2024.04.23


In [46]:
%%writefile make_missing_clrs_heffel.sh
CHROM_SIZES="/tank/projects/tian2023/hg38.reduced.chrom.sizes"
MISS_LIST="/tank/projects/polycomb/results/2024.04.23/missing_coolers.heffel.txt"

COUNT=0
while IFS=" " read -r CONTACT_FPATH CLR_FPATH REMAINDER
do
    echo ${CONTACT_FPATH}
    echo ${CLR_FPATH}
    echo ${COUNT}
    cooler cload pairs --assembly hg38 -c1 1 -p1 2 -c2 3 -p2 4 "${CHROM_SIZES}:10000" \
        --chunksize 150000000 \
        "${CONTACT_FPATH}" \
        "${CLR_FPATH}"
    COUNT=${COUNT}+1
done < "${MISS_LIST}"

Overwriting make_missing_clrs_heffel.sh


In [50]:
clr = cooler.Cooler("/tank/projects/polycomb/data/heffel/cools/2-1-D2-P2.10kb.cool")
clr.pixels()[:]['count'].sum()

120734